# Generate Molecular Structure Descriptors for a Zeolite

Nanoporous materials such as zeolites have pore dimensions similar to that of individual molecules and are used widely in industry as adsorbents, catalysts and chemical separation membranes. The nanoscale cavities in these materials serve as shape and size selective sites facilitating chemical reactions as well as storage, while the channels serve as molecular sieves that can be used for gas separations replacing energetically less efficient distillation processes. 

This notebook presents an approach outlined in [1] for generating computationally efficient digital representations of the molecular structure of nanoporous materials that are then used to compute a number of geometric and statistical descriptors for pore structures. The described methods are capable of identifying and labeling the transport relevant accessible regions in the porous crystals for any user-defined non-spherical atomic-scale morphology. These descriptors can be used as predictors for transport properties.

The notebooks includes the following steps,

 1. [load a cif file with the Zeolite structure](#Load-Structure-of-Interest)
 1. [generate a voxelized representation of the molecular structure](#Generate-Voxelized-Representation-of-the-Pore-Structure)
 1. [compute conventional pore metrics](#Compute-Conventional-Pore-Metrics---PLD-and-LCD)
 1. [compute transport channels through the pore structure](#Geometric-and-Statistical-analysis-of-diffusion-pathways)

![image of zeolite](./DDR_structure.gif)

The image shows the 277x240x813 voxelized molecular structure of the unit cell of a 3D bulk zeolite (namely DDR) at a grid resolution of 0.1 Å. Red voxels correspond to oxygen and orange voxels correspond to silicon atoms.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")
import os
import ase
import time
import numpy as np
import ase.io as aio
import scipy.io as sio
import matplotlib.pyplot as plt
import pymks.atommks.porosity as pore
import pymks.atommks.correlations as corr
from pymks.atommks.grid_generator import generate_grids
from pymks.atommks.helpers import write2vtk
from toolz.curried import pipe, curry, compose
import pandas as pd
from pathlib import Path

np.set_printoptions(precision=1)

## Load the Zeolite Structure

In the following steps we use the `get_structure_data` function to load the ASE atom object and the corresponding atomic radii. The `get_radius` function loads the atom radii from the [Cambridge Crystlaographic Structural Database][cam]. This is used to give each atom a spherical volume in the voxelized representation of the structure.

[cam]: https://www.ccdc.cam.ac.uk/support-and-resources/ccdcresources/Elemental_Radii.xlsx

In [3]:
def get_radius(atom_id, radius_type="vdw"):
    """
    Get the radius of the atom
    
    Args:
      atom_id: element symbol
      radius_type = "vdw" for Van der Waals or "cov" for Covalent
      
    Returns:
      the atomic radius
      
    >>> get_radius('Na')
    2.27
    """
    xl = pd.ExcelFile("Elemental_Radii.xlsx")
    df = xl.parse(sheet_name=0, header = 2, index_col=1)
    
    if radius_type is "cov":
        key = 6
    elif radius_type is "vdw":
        key = 7
    else:
        raise ValueError("radius_type not supported")
    if atom_id in df.index:
        return df.loc[atom_id][key]
    else:
        raise ValueError("Elemental symbol not found")

In [4]:
def get_structure_data(cif_file_path, resize_unit_cell=1):
    """
    Get the ASE atom object (a molecule in many cases) and corresponding
    radii for each atom in the molecule
    
    Args:
      cif_file_path: path to the CIF file
      resize_unit_cell: allows a resize of the atom object
      
    Returns:
      a tuple of the ASE atom object and dictionary of atom radii
    
    >>> get_structure_data('iza_zeolites/DDR.cif')[0].get_cell_lengths_and_angles()
    array([ 27.59,  27.59,  81.5 ,  90.  ,  90.  , 120.  ])
    
    """
    ase_atom = aio.read(cif_file_path).repeat(resize_unit_cell if hasattr(resize_unit_cell, "__len__") else [resize_unit_cell] * 3)
    atom_ids = sorted(np.unique(ase_atom.get_chemical_symbols()))
    return (
        ase_atom,
        {idx:get_radius(idx) for idx in atom_ids}
    )

In [5]:
file_path = "../../../../structures/zz_LEVff-[1,1,0]-L-0.235821_0-U-0.758686_0-ss-19.309257903.cif"
cif = file_path.split("/")[-1][:-4]
ase_atom, radii = get_structure_data(Path(file_path), [2, 2, 1])

# file_path = "iza_zeolites/MFI.cif"
# cif = file_path.split("/")[-1][:-4]
# ase_atom, radii = get_structure_data(Path(file_path), 2)

The ASE atom object

In [6]:
ase_atom

Atoms(symbols='H144O1368Si648', pbc=True, cell=[[45.62710066, 0.0, 0.0], [0.0, 44.58580622, 0.0], [0.0, -69.64149982743176, 40.2092872550728]], spacegroup_kinds=...)

The atomic radii of the atoms types in the structure.

In [7]:
radii

{'H': 1.09, 'O': 1.52, 'Si': 2.1}

Number of atoms in the structure.

In [8]:
len(ase_atom)

2160

## Generate Voxelized Representation of the Pore Structure

The `generate_grids` function generates a voxelized representation of the structure. It returns a dictionary of grids with each grid representing a possible state of the system. Each voxel can only be in one of these states. `n_pixel` represents the number of pixels in a unit length defined used in `ase_atom` (generally Å).

In [9]:
grid_data = generate_grids(
    ase_atom,
    n_pixel=10,
    atomic_radii=radii,
    extend_boundary_atoms=False,
    use_fft_method=False
)

The keys represent the possible states of the system. Here we have `pores` for empty voxels, `O` for oxygen and `Si` for Silicon.

In [10]:
grid_data.keys()

dict_keys(['pores', 'n_pixel', 'H', 'O', 'Si'])

The size of the grids are 277x240x813 voxels, which is sufficent to capture the 2x2x2 sized representation at the resolution of 0.1 Å (`n_pixel` defines 10 pixels per Å)

In [11]:
grid_data['pores'].shape

(442, 438, 204)

## Compute Conventional Pore Metrics - PLD and LCD

Here we compute two pore metrics, the pore limiting diameter (PLD) and the largest cavity diameter (LCD). The PLD refers to the maximum size of a molecule that can pass through the structure in a particular direction.

We use the `calc_pore_metrics` function to calculate the pore metrics. Internally, this uses the Euclidean distance from the pore phase to the nearest atom. This calculation is direction dependent as the PLD 
calculation simulates a probe molecule traversing the structure in a particular direction (by default the direction is assumed to be the last axis, i.e. z-direction for 3D structures).

In [12]:
metrics = pore.calc_pore_metrics(grid_data['pores'], n_pixel=grid_data['n_pixel'])

In [13]:
metrics

{'pld': 2.8984375,
 'lcd': 6.62117862701416,
 'asa': 19446.270000000004,
 'av': 11304.258000000003}

The PLD values will be different in the x-direction for example (`axis=0` signifies the x-direction).

In [14]:
metrics_x = pore.calc_pore_metrics(grid_data['pores'], n_pixel=grid_data['n_pixel'], axis=0)

In [15]:
metrics_x

{'pld': 2.0546875,
 'lcd': 6.206448554992676,
 'asa': 20371.970000000005,
 'av': 10612.910000000002}

## Compute unique diffusion pathways through the structure

In [16]:
%%time
dists, paths, torts_list, indxs_list = pore.calc_diffusion_paths(grid_data["pores"], 
                                                                 r_probe=0.5, 
                                                                 n_pixel=grid_data["n_pixel"])

CPU times: user 1min 50s, sys: 2.04 s, total: 1min 53s
Wall time: 1min 52s


In [17]:
indxs = indxs_list[0]
dlist = [dists[indxs] for indxs in indxs_list]
plds = [dists[indxs].min()*2 for indxs in indxs_list]

In [18]:
def generate_tubular_paths(data, thickness=5):
    
    import scipy
    
    ix = thickness
    
    data = np.pad(
            data,
            ((ix, ix), (ix, ix), (ix, ix)),
            'constant',
            constant_values=0
    )
    
    return scipy.ndimage.filters.convolve(data, 
                                          np.ones((thickness,)*3), 
                                          mode="constant", 
                                          cval=0)[ix:-ix,ix:-ix,ix:-ix]

In [19]:
tubes = generate_tubular_paths((paths>0)*1)

In [20]:
%%time
write2vtk(tubes, "%s_tube.vtk" % cif)

CPU times: user 15.3 s, sys: 1.37 s, total: 16.6 s
Wall time: 18.3 s


* [link1](https://stackoverflow.com/questions/65059923/how-to-generate-itertools-product-without-duplicates-and-symmetric)  
* [link2](https://stackoverflow.com/questions/29314372/itertools-product-without-repeating-duplicates)  
* [link3](https://stackoverflow.com/questions/37186223/applying-two-parameter-function-to-list-to-produce-symmetric-matrix-with-numpy)

In [81]:
%%time
l=10
calc_path_distances_matrix(paths1=dlist[:l], n_workers=1)

CPU times: user 10.9 ms, sys: 62 µs, total: 11 ms
Wall time: 10 ms


array([[ 0. ,  0.6, 13.8, 39.9, 14.3, 40.1, 40. , 40.5, 48.6, 49.4],
       [ 0.6,  0. , 13.2, 40.4, 13.7, 41.6, 40.6, 41.1, 48.3, 49.1],
       [13.8, 13.2,  0. , 30. ,  0.1, 30.8, 23. , 23.1, 63.2, 64.2],
       [39.9, 40.4, 30. ,  0. , 30.1,  0.1, 20.2, 20.3, 32.6, 32.5],
       [14.3, 13.7,  0.1, 30.1,  0. , 30.7, 23. , 23. , 48.7, 64.4],
       [40.1, 41.6, 30.8,  0.1, 30.7,  0. , 20.1, 20. , 32.9, 32.7],
       [40. , 40.6, 23. , 20.2, 23. , 20.1,  0. ,  0.1, 26.6, 26.4],
       [40.5, 41.1, 23.1, 20.3, 23. , 20. ,  0.1,  0. , 26.9, 26.7],
       [48.6, 48.3, 63.2, 32.6, 48.7, 32.9, 26.6, 26.9,  0. ,  0.1],
       [49.4, 49.1, 64.2, 32.5, 64.4, 32.7, 26.4, 26.7,  0.1,  0. ]])

In [23]:
# %%time
# l = 100
# n_workers = 1
# dmat2 = np.zeros((l, l))
# for i0, (ix, iy) in enumerate(combinations(range(l), r=2)):
#     out = dtw_distance(dlist[ix], dlist[iy])
#     dmat2[ix, iy] = out
#     dmat2[iy, ix] = out